In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
!pip install xgboost
!pip install prophet
import statsmodels.api as sm


# Para análisis numérico y clustering
from sklearn.cluster import KMeans
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
import xgboost as xgb
from prophet import Prophet
import warnings
warnings.filterwarnings("ignore")

In [32]:
df_productos = pd.read_excel("C:/Users/marib/OneDrive/Escritorio/Documentos/CasoIBP/Product Review-Portafolio/IBPFinal.xlsx")

In [33]:
df_productos.head()

,Canal,Cliente,Producto ID,Producto,Categoria,Origen,Fecha,Responsable,Precio,Budget,...,Forecast 2022,Forecast 2023,Forecast Consensuado,Forecast Demanda,Forecast Mes Anterior,Forecast Ventas,Ordenes,Ordenes 2022,Ordenes 2023,Pronostico estadistico
0,Canal1,Cliente1,1,Producto1,Categoria1,Base,01/01/2024,Daniel,984,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Canal1,Cliente1,1,Producto1,Categoria1,Base,01/02/2024,Daniel,984,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Canal1,Cliente1,1,Producto1,Categoria1,Base,01/03/2024,Daniel,984,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Canal1,Cliente1,1,Producto1,Categoria1,Base,01/04/2024,Daniel,984,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Canal1,Cliente1,1,Producto1,Categoria1,Base,01/05/2024,Daniel,984,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
df_productos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209100 entries, 0 to 209099
Data columns (total 24 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Canal                   209100 non-null  object 
 1   Cliente                 209100 non-null  object 
 2   Producto ID             209100 non-null  int64  
 3   Producto                209100 non-null  object 
 4   Categoria               209100 non-null  object 
 5   Origen                  209100 non-null  object 
 6   Fecha                   209100 non-null  object 
 7   Responsable             209100 non-null  object 
 8   Precio                  209100 non-null  int64  
 9   Budget                  209100 non-null  float64
 10  Facturado               209100 non-null  float64
 11  Facturado + Pronostico  209100 non-null  float64
 12  Facturado 2022          209100 non-null  float64
 13  Facturado 2023          209100 non-null  float64
 14  Forecast 2022       

In [35]:
#Preprocessamiento
df_productos.columns = df_productos.columns.str.strip().str.replace(" ", "_")
df_productos["Producto_ID"] = df_productos["Producto_ID"].astype(str)
df_productos['Fecha'] = pd.to_datetime(df_productos['Fecha'], dayfirst = True, errors='coerce')
df_productos['Precio']=df_productos['Precio'].astype(float)
df_productos["Budget"] = df_productos["Budget"].astype(int)
df_productos["Facturado"] = df_productos["Facturado"].astype(int)
df_productos["Facturado_+_Pronostico"] = df_productos["Facturado_+_Pronostico"].astype(int)
df_productos["Facturado_2022"] = df_productos["Facturado_2022"].astype(int)
df_productos["Facturado_2023"] = df_productos["Facturado_2023"].astype(int)
df_productos["Forecast_2023 "] = df_productos["Forecast_2023"].astype(int)
df_productos["Forecast_Consensuado "] = df_productos["Forecast_Consensuado"].astype(int)
df_productos["Forecast_Demanda"] = df_productos["Forecast_Demanda"].astype(int)
df_productos["Forecast_Mes_Anterior"] = df_productos["Forecast_Mes_Anterior"].astype(int)
df_productos["Forecast_Ventas"] = df_productos["Forecast_Ventas"].astype(int)
df_productos["Ordenes"] = df_productos["Ordenes"].astype(int)
df_productos["Ordenes_2022"] = df_productos["Ordenes_2022"].astype(int)
df_productos["Ordenes_2023"] =df_productos["Ordenes_2023"].astype(int)
df_productos["Pronostico_estadistico"] = df_productos["Pronostico_estadistico"].astype(int)

In [36]:
df_productos['year'] = df_productos['Fecha'].dt.year
df_productos['month'] = df_productos['Fecha'].dt.month
df_productos['ym'] = df_productos['Fecha'].dt.to_period('M').dt.to_timestamp()
df_productos['quarter'] = df_productos['Fecha'].dt.quarter

In [37]:
df_productos

,Canal,Cliente,Producto_ID,Producto,Categoria,Origen,Fecha,Responsable,Precio,Budget,...,Ordenes,Ordenes_2022,Ordenes_2023,Pronostico_estadistico,Forecast_2023,Forecast_Consensuado,year,month,ym,quarter
0,Canal1,Cliente1,1,Producto1,Categoria1,Base,2024-01-01,Daniel,984.0,0,...,0,0,0,0,0,0,2024,1,2024-01-01,1
1,Canal1,Cliente1,1,Producto1,Categoria1,Base,2024-02-01,Daniel,984.0,0,...,0,0,0,0,0,0,2024,2,2024-02-01,1
2,Canal1,Cliente1,1,Producto1,Categoria1,Base,2024-03-01,Daniel,984.0,0,...,0,0,0,0,0,0,2024,3,2024-03-01,1
3,Canal1,Cliente1,1,Producto1,Categoria1,Base,2024-04-01,Daniel,984.0,0,...,0,0,0,0,0,0,2024,4,2024-04-01,2
4,Canal1,Cliente1,1,Producto1,Categoria1,Base,2024-05-01,Daniel,984.0,0,...,0,0,0,0,0,0,2024,5,2024-05-01,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209095,Canal9,Cliente110,198,Producto198,Categoria2,ajuste fcst local,2024-08-01,Diego,1269.0,0,...,0,0,0,0,0,0,2024,8,2024-08-01,3
209096,Canal9,Cliente110,198,Producto198,Categoria2,ajuste fcst local,2024-09-01,Diego,1269.0,0,...,0,0,0,0,0,0,2024,9,2024-09-01,3
209097,Canal9,Cliente110,198,Producto198,Categoria2,ajuste fcst local,2024-10-01,Diego,1269.0,0,...,0,0,0,0,0,0,2024,10,2024-10-01,4
209098,Canal9,Cliente110,198,Producto198,Categoria2,ajuste fcst local,2024-11-01,Diego,1269.0,0,...,0,0,0,0,0,0,2024,11,2024-11-01,4


Revisión de Producto / Portfolio (Product Review)
📌 Objetivo de la etapa: Revisión de Portafolio Detectar productos que:

Generan más ingresos o unidades.
Tienen baja rotación o alta variabilidad.
Requieren decisiones estratégicas (descontinuar, promover, ajustar precios, etc.).

In [39]:
df_productos = df_productos.copy()
df_productos['Facturado'] = df_productos['Facturado'].clip(lower=0)
df_productos['Precio'] = df_productos['Precio'].clip(lower=0)
df_productos['Ingreso'] = df_productos['Facturado'] * df_productos['Precio']


#Agrupación por producto.

df_agrupado = df_productos.groupby('Producto').agg({'Facturado': 'sum', 'Ingreso':'sum'}).rename(
    columns={'Facturado': 'Unidades Totales', 'Ingreso': 'Ingresos Totales'}).sort_values('Ingresos Totales', ascending=False)

#Calculo de % acumulado para analisis de pareto
df_agrupado['% Acumulado Ingresos'] = df_agrupado['Ingresos Totales'].cumsum() / df_agrupado['Ingresos Totales'].sum()
df_agrupado['% Acumulado Unidades'] = df_agrupado['Unidades Totales'].cumsum() / df_agrupado['Unidades Totales'].sum()


#Clasificación ABC ingreso
df_agrupado['Clasificación ABC_ingreso'] = pd.cut(df_agrupado['% Acumulado Ingresos'], bins=[0,0.8,0.95,1.01], labels=['A','B','C'])
df_agrupado['Clasificación ABC_unidades'] = pd.cut(df_agrupado['% Acumulado Unidades'], bins=[0,0.8,0.95,1.01], labels=['A','B','C'])

#Matriz cruzada
df_agrupado['Clasificación Cruzada'] = df_agrupado['Clasificación ABC_ingreso'].astype(str) + " - " + df_agrupado['Clasificación ABC_unidades'].astype(str)
df_agrupado

#df_agrupado.reset_index().to_excel("archivo.xlsx", index=False)



,Unidades Totales,Ingresos Totales,% Acumulado Ingresos,% Acumulado Unidades,Clasificación ABC_ingreso,Clasificación ABC_unidades,Clasificación Cruzada
Producto,,,,,,,
Producto165,1218649,962173246.0,0.139314,0.108134,A,A,A - A
Producto157,433307,564035437.0,0.220981,0.146582,A,A,A - A
Producto114,498300,330802820.0,0.268878,0.190797,A,A,A - A
Producto115,374510,327941960.0,0.316361,0.224029,A,A,A - A
Producto211,354500,253467500.0,0.353060,0.255484,A,A,A - A
...,...,...,...,...,...,...,...
Producto279,0,0.0,1.000000,1.000000,C,C,C - C
Producto278,0,0.0,1.000000,1.000000,C,C,C - C
Producto277,0,0.0,1.000000,1.000000,C,C,C - C


In [40]:
df_agrupado.to_excel("clasificación_cruzada.xlsx", index=False)

In [41]:
# Elasticidad Precio-Demanda
#📌 Objetivo: medir qué tan sensible es la demanda de cada producto a cambios en el precio.

resultados = []
for prod in df_productos['Producto'].unique():
    sub = df_productos[df_productos['Producto'] == prod]
    #Evitar problemas log(0)
    sub = sub[(sub['Precio'] > 0) & (sub['Facturado'] > 0)]

    sub['log_precio'] = np.log(sub['Precio'])
    sub['log_Facturado'] = np.log(sub['Facturado'])

    X = sm.add_constant(sub['log_precio'])
    y = sub['log_Facturado']

    if len(sub) > 2:
        
        modelo = sm.OLS(y,X).fit()
        elasticidad = modelo.params['log_precio']
        unidades_totales = sub['Facturado'].sum()
        resultados.append({'Producto':prod, 'Elasticidad_Precio': elasticidad, 'unidades_totales':unidades_totales})

elasticidad_df = pd.DataFrame(resultados)
elasticidad_df

#elasticidad_df.to_csv("elasticidad_precio.csv", index=False)


#Interpretación:

#Elasticidad < -1 → muy sensible al precio (pequeños cambios generan grandes variaciones en demanda).
#Entre -1 y 0 → poco sensible.
#Positiva → caso raro (posible bien de lujo o error en datos).



,Producto,Elasticidad_Precio,unidades_totales
0,Producto1,-11.008107,10450
1,Producto2,10.841383,3742
2,Producto4,-20.440295,2113
3,Producto5,-15.130173,727
4,Producto6,-0.154570,5745
...,...,...,...
164,Producto229,0.868895,1750
165,Producto230,1.074213,10560
166,Producto241,1.018799,12625
167,Producto245,-2.748146,81518


In [42]:
#Identificación de Productos Canibalizadores
#📌 Objetivo: encontrar productos que al aumentar su venta, reducen la de otros (competencia interna).

df_productos.copy()
from itertools import combinations

#Matriz de ventas por periodo y producto
df_productos = df_productos.pivot_table(values = 'Facturado', index='Fecha', columns='Producto', aggfunc='sum').fillna(0)

canibalizacion = []
for prod1, prod2 in combinations(df_productos.columns, 2):
    correlacion = df_productos[prod1].corr(df_productos[prod2])
    if correlacion < -0.5:
        canibalizacion.append({'Producto_A': prod1, 'Producto_B':prod2, 'Correlacion':correlacion})

canibal_df = pd.DataFrame(canibalizacion)
canibal_df
#canibal_df.to_csv('canibalizacion.csv', index=False)



""


In [43]:
#Segmentación de Productos por Rentabilidad y Crecimiento (BCG + AI)
#📌 Objetivo: agrupar productos en Estrellas, Vacas, Interrogantes, Perros usando ML.

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

df_productos = pd.read_excel("C:/Users/marib/OneDrive/Escritorio/Documentos/CasoIBP/Product Review-Portafolio/IBPFinal.xlsx")

df_metrics = df_productos.groupby('Producto').agg({
    'Facturado': 'sum',
    'Precio': 'mean'
}).reset_index()

#Calculo de ingreso total y crecimiento 
df_metrics['Ingreso'] = df_metrics['Facturado'] * df_metrics['Precio']
df_metrics['Crecimiento'] = np.random.rand(len(df_metrics))

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_metrics[['Ingreso', 'Crecimiento']])
Kmeans  = KMeans(n_clusters=4, random_state=42)
df_metrics['Cluster'] = Kmeans.fit_predict(X_scaled)
df_metrics

#df_metrics.to_csv("segmentacion_bcg.csv", index=False)
    

,Producto,Facturado,Precio,Ingreso,Crecimiento,Cluster
0,Producto1,10450.0,973.444444,1.017249e+07,0.400574,0
1,Producto10,4096.0,464.140625,1.901120e+06,0.374151,0
2,Producto100,50064.0,190.129032,9.518620e+06,0.987838,1
3,Producto101,4460.0,117.522936,5.241523e+05,0.134750,3
4,Producto102,5850.0,1045.276596,6.114868e+06,0.666446,1
...,...,...,...,...,...,...
527,Producto962,0.0,89.000000,0.000000e+00,0.101844,3
528,Producto97,2850.0,1231.000000,3.508350e+06,0.044315,3
529,Producto978,0.0,244.000000,0.000000e+00,0.863264,1
530,Producto98,155142.0,1127.000000,1.748450e+08,0.450428,0


In [44]:
df_productos['Facturado'].describe()

count    209100.000000
mean         53.313152
std        3516.627606
min     -121076.720000
25%           0.000000
50%           0.000000
75%           0.000000
max      879200.000000
Name: Facturado, dtype: float64

In [45]:
#Análisis de Erosión de Margen
#📌 Objetivo: detectar si las promociones o descuentos realmente generan rentabilidad.

df_productos['Costo'] = df_productos['Precio'] * 0.6 # Promoción del 60% del precio
df_productos['Margen'] = (df_productos['Precio'] - df_productos['Costo'] * df_productos['Facturado'])

erosion = df_productos.groupby('Producto').agg({
    'Margen':'sum',
    'Facturado':'sum'
}).reset_index()

#erosion.to_csv("erosion_margen.csv", index=False)

erosion
       

,Producto,Margen,Facturado
0,Producto1,-4841679.0,10450.0
1,Producto10,265495.2,4096.0
2,Producto100,-5528898.0,50064.0
3,Producto101,-164094.0,4460.0
4,Producto102,-2487678.0,5850.0
...,...,...,...
527,Producto962,3204.0,0.0
528,Producto97,-1455660.0,2850.0
529,Producto978,5856.0,0.0
530,Producto98,-104914203.6,155142.0


In [46]:
#Desempeño por Categoría y Responsable
#📌 Objetivo: Evaluar cómo cada categoría y cada responsable están contribuyendo al forecast accuracy y a las ventas.

#Calculo de MAPE y BIAS por Categoría y Responsable
df_desempeño = pd.read_excel("C:/Users/marib/OneDrive/Escritorio/Documentos/CasoIBP/Product Review-Portafolio/IBPFinal.xlsx").fillna(0)

df_desempeño = df_desempeño[df_desempeño['Facturado'] > 0].copy()

df_desempeño['Error'] = df_desempeño['Facturado']  - df_desempeño['Forecast Consensuado']
df_desempeño['APE'] = abs(df_desempeño['Error']) / df_desempeño['Facturado']
df_desempeño['Bias']= df_desempeño['Error'] / df_desempeño['Facturado']

result = df_desempeño.groupby(['Categoria', 'Responsable']).agg(
    Ventas_Real=('Facturado','sum'),
    Forecast = ('Forecast Consensuado','sum'),
    MAPE = ('APE', 'mean'),
    Bias = ('Bias','mean')

).reset_index()

result['MAPE'] = (result['MAPE'] * 100).round(2)
result['Bias'] = (result['Bias'] * 100).round(2)

result['Num_Productos'] = df_desempeño.groupby(['Categoria', 'Responsable'])['Producto'].nunique().values


#result.to_csv("Desempeño.cat.resp.csv", index=False)

result


,Categoria,Responsable,Ventas_Real,Forecast,MAPE,Bias,Num_Productos
0,Categoria1,Alex,225.000000,125.000000,50.00,50.00,2
1,Categoria1,Artesanos,2659.018946,2301.346717,35.18,24.81,6
2,Categoria1,Dani,960.000000,700.000000,84.23,-44.94,2
3,Categoria1,Daniel,13565.917400,12963.614246,65.11,-17.14,8
4,Categoria1,Dolores,14325.000000,16617.000000,221.48,-178.06,4
...,...,...,...,...,...,...,...
183,Categoria9,Manuel,109268.000000,94602.979167,84.02,-2.57,28
184,Categoria9,Pablo,167864.572146,155674.196067,62.51,1.43,32
185,Categoria9,Paola,211365.000000,208286.977316,21.68,-6.14,13
186,Categoria9,Renata,11645.000000,20919.158579,137.94,-71.27,2


In [47]:
#Análisis por cliente
#Objetivo: Identificar clientes clave, problemas de servicio y oportunidades.

resumen_cliente = df_desempeño.groupby('Cliente').agg(
    MAPE = ('APE','mean'),
    Ventas_Totales=('Facturado','sum'),
    Precio_Promedio=('Precio','mean')
).reset_index()

top_clientes_criticos = resumen_cliente[
    (resumen_cliente['Ventas_Totales'] > resumen_cliente['Ventas_Totales'].quantile(0.75)) &
    (resumen_cliente['MAPE']> resumen_cliente['MAPE'].median())

]

print(top_clientes_criticos)

#top_clientes_criticos.to_csv("topclientes.csv", index=False)


       Cliente      MAPE  Ventas_Totales  Precio_Promedio
0    Cliente10  0.877674    3.962962e+05       822.069767
11   Cliente11  1.117910    6.023908e+05       779.317708
12  Cliente111  1.000000    1.767302e+06       837.608696
14   Cliente13  1.555078    1.225672e+05       839.432258
15   Cliente14  1.014082    3.996766e+05       758.507143
16   Cliente15  1.440209    3.958197e+05       754.830409
17   Cliente18  1.077449    1.342769e+05       739.587879
18   Cliente19  4.078749    1.326591e+05       777.462121
19    Cliente2  1.426562    1.029287e+05       797.562963
21   Cliente22  0.841221    1.668664e+05       729.695455
39    Cliente5  0.934247    9.945381e+04       723.420635
60    Cliente8  1.044312    1.643564e+05       739.812155
67    Cliente9  1.587794    1.385191e+06       779.749361


In [48]:
# Clasificación simbólica por desempeño
def clasificar_prioridad(row):
    if row['MAPE'] < 1 and row['Ventas_Totales'] > resumen_cliente['Ventas_Totales'].median():
        return '🟢 Alta'
    elif row['MAPE'] < 2:
        return '🟠 Media'
    else:
        return '🔴 Baja'

# Aplicar clasificación
resumen_cliente['Prioridad'] = resumen_cliente.apply(clasificar_prioridad, axis=1)
resumen_cliente

#resumen_cliente.to_csv("resumen_cliente.csv", index=False)

,Cliente,MAPE,Ventas_Totales,Precio_Promedio,Prioridad
0,Cliente10,0.877674,396296.239600,822.069767,🟢 Alta
1,Cliente100,0.766031,62576.537166,947.285714,🟢 Alta
2,Cliente101,0.621246,4587.820000,914.000000,🟠 Media
3,Cliente102,0.524170,882.000000,719.800000,🟠 Media
4,Cliente103,0.469951,79875.000000,1209.545455,🟢 Alta
...,...,...,...,...,...
72,Cliente95,0.527003,33995.148928,997.162162,🟢 Alta
73,Cliente96,0.213464,47783.887546,795.500000,🟢 Alta
74,Cliente97,0.533434,31855.000000,661.687500,🟢 Alta
75,Cliente98,0.531637,53999.978570,742.666667,🟢 Alta


In [49]:
#Análisis de Ciclo de Vida de Producto (PLC – Product Life Cycle)
#Objetivo en IBP
#Identificar en qué etapa se encuentra cada producto (Introducción, Crecimiento, Madurez o Declive) para:
#Ajustar estrategia de forecast (p. ej., no proyectar crecimiento en madurez o declive).
#Definir tácticas de precio y promoción.
#Priorizar inversión en producción y marketing.
#Tomar decisiones de descontinuación o relanzamiento.

import pandas as pd
import numpy as np

#impiar datos
# Eliminar filas con Facturado <= 0 o Facturado 2023 <= 0 (ajustes o devoluciones)
df_limpio = df_productos[
    (df_productos['Facturado'] > 0) & 
    (df_productos['Facturado 2023'] >= 0) &  # permitimos 0 en 2023 para detectar nuevos productos
    (df_productos['Facturado 2022'] >= 0)   # permitimos 0 en 2022 también
].copy()

# Agrupar ventas por Producto ID y año
# Agrupamos las ventas por Producto ID sumando Facturado (2024), Facturado 2023 y Facturado 2022
ventas_agg = df_limpio.groupby('Producto ID')[['Facturado', 'Facturado 2023', 'Facturado 2022']].sum().reset_index()

#3. Definir etapa estratégica básica
def asignar_etapa_estrategica(row):
    if row['Facturado'] > 0 and row['Facturado 2023'] == 0:
        return 'Nuevo 2024'
    elif row['Facturado'] > 0 and row['Facturado 2023'] > 0:
        return 'Activo'
    elif row['Facturado'] == 0 and row['Facturado 2023'] > 0:
        return 'Ajuste o devolución'
    else:
        return 'Descontinuado'

ventas_agg['Etapa Estratégica'] = ventas_agg.apply(asignar_etapa_estrategica, axis=1)

# Filtrar solo productos activos o nuevos
ventas_agg = ventas_agg[ventas_agg['Etapa Estratégica'].isin(['Activo', 'Nuevo 2024'])].copy()

# Calcular Variación % de ventas entre 2024 y 2023
def calcular_variacion(row):
    if row['Facturado 2023'] == 0:
        return 100.0  # Crecimiento 100% para nuevos productos sin ventas previas
    else:
        return ((row['Facturado'] - row['Facturado 2023']) / row['Facturado 2023']) * 100

ventas_agg['Variación %'] = ventas_agg.apply(calcular_variacion, axis=1)

# Clasificar etapa del ciclo de vida
def clasificar_ciclo_vida(row):
    if row['Facturado 2023'] == 0 and row['Facturado'] > 0:
        return 'Introduccion'
    elif row['Variación %'] > 20:
        return 'Crecimiento'
    elif row['Variación %'] < -20:
        return 'Declive'
    else:
        return 'Madurez'

ventas_agg['Ciclo de Vida'] = ventas_agg.apply(clasificar_ciclo_vida, axis=1)

# Mostrar resultado final
print(ventas_agg[['Producto ID', 'Facturado 2022', 'Facturado 2023', 'Facturado', 'Etapa Estratégica', 'Variación %', 'Ciclo de Vida']])


#ventas_agg.to_excel('ciclo_vida_productos.xlsx', index=False)


     Producto ID  Facturado 2022  Facturado 2023     Facturado  \
0              1            0.00            0.00  10450.000000   
1              2            0.00            0.00   3753.466987   
2              4            0.00            0.00   2117.500000   
3              5            0.00            0.00    730.000000   
4              6            0.00            0.00   5745.000000   
..           ...             ...             ...           ...   
226          272            0.00            0.00  14825.000000   
227          273            0.00            0.00  13525.000000   
228          274            0.00            0.00  12975.000000   
229         2409         1111.32         1292.76    600.000000   
230         2411         1050.00         1500.00    850.000000   

    Etapa Estratégica  Variación % Ciclo de Vida  
0          Nuevo 2024   100.000000  Introduccion  
1          Nuevo 2024   100.000000  Introduccion  
2          Nuevo 2024   100.000000  Introduccion  
3  

In [74]:
with pd.ExcelWriter('Portafolio.xlsx') as writer:
    df_agrupado.reset_index().to_excel("clasificacion_cruzada.xlsx", index=False)
    elasticidad_df.to_excel("elasticidad_precio.xlsx", index=False)
    canibal_df.to_excel('canibalizacion.xlsx', index=False)
    df_metrics.to_excel("segmentacion_bcg.xlsx", index=False)
    erosion.to_excel("erosion_margen.xlsx", index=False)
    result.to_excel("Desempeño.cat.resp.xlsx", index=False)
    top_clientes_criticos.to_excel("topclientes.xlsx", index=False)
    resumen_cliente.to_excel("resumen_cliente.xlsx", index=False)
    ventas_agg.to_excel('ciclo_vida_productos.xlsx', index=False)
